In [1]:
# Install the relevant packages

import os

In [2]:
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    deployment_name="gpt-35-turbo-16k",
    openai_api_key=api_key,
    openai_api_type=endpoint,
    temperature=0.7,
)

In [3]:
llm.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm an AI language model, so I don't have feelings, but I'm here to help you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-7fdd9b8a-397a-4ba3-8d7c-0f3b8a504ef6-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44})

In [4]:
# Now we need to read our pdf file

from langchain_community.document_loaders import PyPDFLoader

pdf_filepath = "content/document.pdf"

pdf_loader = PyPDFLoader(pdf_filepath)

pdf_text = pdf_loader.load()

In [5]:
print(f" The number of documents : {len(pdf_text)}")
print(f" The number of words in the document/s : {sum([len(x.page_content.split()) for x in pdf_text])}")
print(f" The number of tokens count : {llm.get_num_tokens(pdf_text[0].page_content)}")

 The number of documents : 25
 The number of words in the document/s : 5287
 The number of tokens count : 769


In [14]:
from langchain.document_loaders import DirectoryLoader

directory_pdfs = DirectoryLoader('content', glob="**/*.pdf")
pdf_text = directory_pdfs.load()


In [15]:
from langchain.chains.question_answering import load_qa_chain

def simple_qa_chain(query:str,input_docs:list,llm,verbose=False):
  """
  Return answer to the query
  """
  
  qa_chain = load_qa_chain(llm, chain_type="stuff",verbose=verbose)

  return qa_chain.run(input_documents=input_docs, question=query)

In [16]:
query = "What is the project about?"

print(simple_qa_chain(query,pdf_text[:10],llm))

# Here we set the pages to 10 pages as the number of tokens is too high for the model to handle
# In the next section we will split the document into smaller chunks


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16384 tokens. However, your messages resulted in 25513 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [17]:
# To split the documents, we first need to encode the documents and store it in a vector databse.
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(pdf_text)


In [18]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings()


In [19]:
from langchain.vectorstores import FAISS

faiss = FAISS.from_documents(texts, embeddings)

In [20]:
def qa_compute(query: str, vectorstore, llm, verbose=False):
    """
    Return answer to the query
    """
    input_docs = vectorstore.similarity_search(query, k=3)
    qa_chain = load_qa_chain(llm, chain_type="stuff", verbose=verbose)

    return qa_chain.run(input_documents=input_docs, question=query)

In [21]:
query = "What is the project about?"
qa_compute(query,faiss,llm)

'Based on the provided context, the project appears to be about a website or app called Mobalytics that focuses on the game League of Legends. It offers features such as profiles, GPI (Game Performance Index) analysis, champion information, and a tier list. The project may also include updates on balance changes, such as nerfs to the base stats of champions like Irelia and Jinx.'

In [22]:
def similar_dist(query:str,vectorstore):
  """
  Return Dataframe that consist of the text and the its distance from the query vector
  """

  distance_list = []

  similar_score = vectorstore.similarity_search_with_score(query)

  for n in range(0,len(similar_score)):
      text = vectorstore.similarity_search_with_score(query)[n][0].page_content
      distance  = vectorstore.similarity_search_with_score(query)[n][1]

      distance_list.append((text,distance))

  return pd.DataFrame(distance_list,columns=['text','distance to query'])    

In [25]:
import gradio as gr

/Users/koiljat/Desktop/chat_with_document/venv/lib/python3.11/site-packages/gradio_client/documentation.py:104: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  line = line[4:]
/Users/koiljat/Desktop/chat_with_document/venv/lib/python3.11/site-packages/gradio_client/documentation.py:104: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  line = line[4:]


In [26]:

def source_question_answer(query:str,vectorstore:FAISS=faiss,llm=llm):
  """
  Return answer to the query
  """
  input_docs = vectorstore.similarity_search(query,k=3)
  qa_chain = load_qa_chain(llm, chain_type="stuff")

  response = qa_chain.run(input_documents=input_docs, question=query)
  source_docs_1 =  input_docs[0].page_content
  source_docs_2 =  input_docs[1].page_content
  return response,source_docs_1 ,source_docs_2 


demo = gr.Interface(
    source_question_answer,
    inputs= gr.Textbox(placeholder="Enter query here..."),
    # outputs=["text","text","text"]
    outputs=[gr.Textbox(label="Response"), gr.Textbox(label="Source Text"), gr.Textbox(label="Source Text 2")],
    examples=[
        ["What is the best champion now?"],
        ["What is the best item now?"]],
    title="Chat with Data\n\n Ask me anything about league of legeends"    
)

demo.launch(share=True)
    

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://2077bc7c238279e062.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [27]:
def question_answer(
    query: str, vectorstore: FAISS = faiss, llm: llm = llm
):
    """
    Return answer to the query
    """
    input_docs = vectorstore.similarity_search(query, k=3)
    qa_chain = load_qa_chain(llm, chain_type="stuff")

    response = qa_chain.run(input_documents=input_docs, question=query)
    return response


def chat(chat_history, user_input):

    # input_docs = vectorstore.similarity_search(user_input,k=3)
    # qa_chain = load_qa_chain(llm, chain_type="stuff")
    # chain_response = qa_chain.run(input_documents=input_docs, question=user_input)

    chain_response = question_answer(user_input)

    print(chain_response)
    response = ""
    for letter in "".join(
        chain_response
    ):  # [bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
        response += letter + ""

        yield chat_history + [(user_input, response)]

In [28]:
with gr.Blocks() as demo:

          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this?")
          message.submit(chat, [chatbot,message], chatbot)

demo.queue().launch(debug = True, share=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://a29717901c558eea08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2077bc7c238279e062.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://a29717901c558eea08.gradio.live
